In [1]:
from os import listdir
from os.path import isfile, join
import re
import pandas as pd
from lexnlp.extract.en.entities import nltk_re
from lexnlp.nlp.en.segments import sentences
from lexnlp.utils import parse_df

c:\users\willem jan willemse\anaconda3\envs\solvency2-nlp\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
DATA_PATH = "..\data\interim\sfcr"

In [3]:
txt_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f)) and f[-3:]=='txt']
documents = []
for file_name in txt_files:
    file = open(join(DATA_PATH, file_name), "rb")
    text = file.read().decode('utf-8')
    file.close()
    #text = text.replace("\n", " ")
    documents.append(text)
    
pickle_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f)) and f[-6:]=='pickle']
df = pd.DataFrame()
for file_name in pickle_files:
    df = df.append(pd.read_pickle(join(DATA_PATH, file_name)), ignore_index = True)

In [4]:
print("Number of documents: " + str(len(documents)))
print("Number of sentences: " + str(len(df.index)))

Number of documents: 16
Number of sentences: 46104


In [5]:
from sklearn.externals import joblib
import os

In [6]:
from lexnlp.nlp.en.segments import titles, sections, paragraphs, pages
from src.nlp.en.segments import solvency2_titles
from src.nlp.en.segments import solvency2_document_year

c:\users\willem jan willemse\anaconda3\envs\solvency2-nlp\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 0.19.0 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\willem jan willemse\anaconda3\envs\solvency2-nlp\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator ExtraTreesClassifier from version 0.19.0 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\willem jan willemse\anaconda3\envs\solvency2-nlp\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
for idx in range(len(documents)):
    documents[idx] = documents[idx].replace('\uf0b7', "")
    documents[idx] = documents[idx].replace('\uf0a7', "")
    documents[idx] = documents[idx].replace('\uf0fc', "")
    documents[idx] = documents[idx].replace('\uf00c', "")
    documents[idx] = documents[idx].replace('\uf00d', "")
    documents[idx] = documents[idx].replace('\uf020', "")

In [8]:
# for item in range(len(documents[2:])):
#     print("Document: " + str(txt_files[item]) + " (" + str(item) + ")")
#     print("Sections: " + str(list(sections.get_sections(documents[item], score_threshold=0.5))))

In [9]:
from lexnlp.extract.common import fact_extracting

In [10]:
#a = list(sections.get_sections(documents[14], score_threshold = 0.5))

In [11]:
#a[100]

In [12]:
import src
f = "..//dictionary//en//terminology.csv"
config_list = list(src.extract.en.solvency2_terms.load_entities_dict_by_path(f))

b = fact_extracting.FactExtractor
b.ensure_parser_arguments_en(config_list)

In [13]:
# sent = "There was fifty one of them. On May 5 the Solvency Capital Requirement. is 5 million."
# sent = sent.replace(".", " .")
sent = "Totals 6,576,210.93 Loan 12/11/2007          10,000,000.00"
c = b.parse_text(sent, 'en')

[('6,576,210.93', (6, 20), {'time': [], 'hours': [], 'minutes': [], 'seconds': [], 'microseconds': [], 'time_periods': [], 'timezones': [], 'digits_modifier': [], 'digits': ['6', '576', '210', '93'], 'days': [], 'months': [], 'delimiters': ['', ',', ',', '.'], 'extra_tokens': []}), ('12/11/2007 10,000,000.00', (24, 58), {'time': [], 'hours': [], 'minutes': [], 'seconds': [], 'microseconds': [], 'time_periods': [], 'timezones': [], 'digits_modifier': [], 'digits': ['12', '11', '2007', '10', '000', '000', '00'], 'days': [], 'months': [], 'delimiters': ['', '/', '/', '', ',', ',', '.'], 'extra_tokens': []})]


In [14]:
c

{<AnnotationType.ratio: 20>: [ [ratio] at (25..36), loc: en],
 <AnnotationType.amount: 2>: [ [amount] at (7..20), loc: en,
   [amount] at (45..58), loc: en]}

In [15]:
sent[8:13]

',576,'

In [16]:
import lexnlp

In [17]:
sent = "May 16, 2015   100.0"
#list(lexnlp.extract.common.date_parsing.datefinder.extract_date_strings(sent))

In [18]:
from datetime import datetime
base_date = datetime.now().replace(
        day=1, month=1, hour=0, minute=0, second=0, microsecond=0)

    # Find potential dates
date_finder = lexnlp.extract.en.dates.DateFinder(base_date=base_date)
#list(date_finder.find_dates(sent))
list(lexnlp.extract.en.dates.get_raw_dates(sent))

[('May 16, 2015 100.0', (0, 20), {'time': [], 'hours': [], 'minutes': [], 'seconds': [], 'microseconds': [], 'time_periods': [], 'timezones': [], 'digits_modifier': [], 'digits': ['16', '2015', '100', '0'], 'days': [], 'months': ['May'], 'delimiters': ['', ',', '', '.'], 'extra_tokens': []})]


[]

In [19]:
possible_dates = [(date_string, index, date_props) for date_string, index, date_props in
                      date_finder.extract_date_strings(sent, strict=False)]
possible_dates

[('May 16, 2015 100.0',
  (0, 20),
  {'time': [],
   'hours': [],
   'minutes': [],
   'seconds': [],
   'microseconds': [],
   'time_periods': [],
   'timezones': [],
   'digits_modifier': [],
   'digits': ['16', '2015', '100', '0'],
   'days': [],
   'months': ['May'],
   'delimiters': ['', ',', '', '.'],
   'extra_tokens': []})]

In [20]:
from dateutil.parser import *
from datetime import *
now = parse("May 16, 2016 100")

ValueError: ('Unknown string format:', 'May 16, 2016 100')

In [ ]:
now